In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [2]:
# 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

In [3]:
# 카테고리 리스트
categories = {
    "스킨/토너": "100000100010013",
    "에센스/세럼/앰플": "100000100010014",
    "크림": "100000100010015"
}

In [4]:
def get_all_products(category_id):
    url = f"https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo={category_id}"
    driver.get(url)
    time.sleep(2)  # 로딩 대기
    
    # 무한 스크롤
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        # 상품 리스트 추출
    product_elements = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list li")
    product_links = []
    for elem in product_elements:
        try:
            link = elem.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            product_links.append(link)
        except:
            continue
    return product_links

In [5]:
def get_reviews(driver, product_url):
    driver.get(product_url)
    time.sleep(2)
    
    # 리뷰 탭 클릭
    try:
        review_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "goods_reputation"))
        )
        review_tab.click()
    except:
        print(f"리뷰 탭 없음: {product_url}")
        return []

    time.sleep(2)
    reviews = []

    while True:
        # 현재 페이지 번호 리스트 가져오기
        try:
            page_links = driver.find_elements(By.CSS_SELECTOR, "#gdasContentsArea .pageing a[data-page-no]")
            page_numbers = sorted(set(int(link.get_attribute("data-page-no")) for link in page_links))
        except:
            break

        for page_num in page_numbers:
            try:
                # 페이지 클릭
                page_btn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, f'#gdasContentsArea .pageing a[data-page-no="{page_num}"]'))
                )
                driver.execute_script("arguments[0].click();", page_btn)
                time.sleep(1.5)

                # 리뷰 수집
                review_items = driver.find_elements(By.CSS_SELECTOR, "#gdasList li")
                for item in review_items:
                    try:
                        text = item.find_element(By.CSS_SELECTOR, ".review_cont .txt_inner").text.strip()
                        point = item.find_element(By.CSS_SELECTOR, ".review_cont .point").get_attribute("style")
                        reviews.append({"text": text, "point": point})
                    except:
                        continue

            except Exception as e:
                print(f"[페이지 클릭 실패] page {page_num}: {e}")
                continue

        # 다음 10페이지 묶음 버튼(.next)이 있으면 클릭
        try:
            next_btn = driver.find_element(By.CSS_SELECTOR, "#gdasContentsArea .pageing a.next")
            driver.execute_script("arguments[0].click();", next_btn)
            time.sleep(2)
        except:
            break  # 다음 묶음 없으면 종료

    return reviews


In [6]:
# 누적할 리스트 생성 
all_reviews = [] 

In [7]:
# 전체 실행
for category_name, category_id in categories.items():
    print(f"카테고리: {category_name}")
    product_links = get_all_products(category_id)
    print(f"상품 개수: {len(product_links)}개")

    for link in product_links:
        print(f"상품 링크: {link}")
        reviews = get_reviews(driver,link)
        print(f"리뷰 개수: {len(reviews)}")
        for review in reviews:
            print(f"점수: {review['point']}, 내용: {review['text']}")
            all_reviews.append({
                "카테고리": category_name,
                "상품URL": link,
                "리뷰점수": review['point'],
                "리뷰내용": review['text']
            })

카테고리: 스킨/토너
상품 개수: 20개
상품 링크: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000224583&dispCatNo=100000100010013&trackingCd=Cat100000100010013_Clickbest&curation=like&egcode=c001_c001&rccode=pc_category_01_a&egrankcode=1&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EB%A7%8E%EC%9D%B4%EB%B3%B8%EC%83%81%ED%92%88&t_number=1
리뷰 개수: 40
점수: width: 100%;, 내용: 생각보다 촉촉한 제형이라 속건조에 좋네요!
물토너보다 덜 산뜻하지만 오히려 좋아~
쿨링감은 가짜 민트향 나는 얼굴 차가운 느낌만 나는 가짜쿨링이 아니라 ㄹㅇ 피부 속부터 진정이되는? 느낌이에요
물리적으로 피부가 시원한 느낌은 아닌데 피부 자체를 만져보면 온도가 낮아진 느낌이 나고 열감이 떨어짐
오히려 좋아~
용량도 꽤 많고 논코메도제닉 제품이라 모공 막혀서 트러블 잘 나는 저에겐 인생템 될거같네요… 빌리프 오랜만에 사봤는데 아주 맘에듭니다!!
점수: width: 100%;, 내용: 토너를 히알루 성분이 있는 타입을 좋아하는 것 같아요.
제품에 세라마이드 성분도 들어있는 것 같습니다.
토너에서 보습이 느껴지는 걸 별로 안좋아해서~
시원함은 일시적이라 프로즌 근처는 못가는 거 같구요
어차피 앰플류 바를거라 피부장벽보습은 필요가 없는데
진짜 시원한 성분을 넣어주셨음 좋지 않았을까요.
가격은 조만간 오르겠죠. 다른 대체 토너가 많습니다.
크림이 좋았는데 그냥 그거나 하나 더 살걸 그랬나봐요
점수: width: 100%;, 내용: 배송이 정말 오래걸렸네요 팩토하기 좋아요 쿨링은 사실 많이느껴지지않아요
점수: width: 100%;, 내용: 배송도 빠르고 빌리프 제품치고 가성비

WebDriverException: Message: tab crashed
  (Session info: chrome=136.0.7103.114)
Stacktrace:
	GetHandleVerifier [0x00007FF69311CF45+75717]
	GetHandleVerifier [0x00007FF69311CFA0+75808]
	(No symbol) [0x00007FF692EE8DCC]
	(No symbol) [0x00007FF692ED617B]
	(No symbol) [0x00007FF692ED3E8A]
	(No symbol) [0x00007FF692ED483F]
	(No symbol) [0x00007FF692EE33AE]
	(No symbol) [0x00007FF692EF97E1]
	(No symbol) [0x00007FF692F0091A]
	(No symbol) [0x00007FF692ED4FAD]
	(No symbol) [0x00007FF692EF8FD1]
	(No symbol) [0x00007FF692F8F276]
	(No symbol) [0x00007FF692F67153]
	(No symbol) [0x00007FF692F30421]
	(No symbol) [0x00007FF692F311B3]
	GetHandleVerifier [0x00007FF69341D71D+3223453]
	GetHandleVerifier [0x00007FF693417CC2+3200322]
	GetHandleVerifier [0x00007FF693435AF3+3322739]
	GetHandleVerifier [0x00007FF693136A1A+180890]
	GetHandleVerifier [0x00007FF69313E11F+211359]
	GetHandleVerifier [0x00007FF693125294+109332]
	GetHandleVerifier [0x00007FF693125442+109762]
	GetHandleVerifier [0x00007FF69310BA59+48

In [ ]:
# 6. 저장
df = pd.DataFrame(all_reviews)
df.to_csv("oliveyoung_reviews.csv", index=False, encoding="utf-8-sig")
print("\n✅ CSV 저장 완료: oliveyoung_reviews.csv")

# 7. 브라우저 종료
driver.quit()